In [1]:
import math
import random
from functools import reduce
import numpy as np
import matplotlib.pyplot as plt

In [2]:
"""
初始化种群
"""

class Individual:
    """
    种群个体信息
    """
    var = []
    obj = []
    s = []
    n = 0
    distance  = 0
    rank = 0

def init_pop(pop_size, num_var, num_obj, num_max, num_min, function):
    """
    :num_var: 种群大小
    :num_obj:目标数
    :param pop_dna_lenght: 个体DNA长度
    :return: 返回一个pop_size * pop_dna_lenght的二维数组
    """
    Pop = []
    for i in range(0,pop_size):
        pop = Individual()
        pop.id = i
        pop.var = [num_min + (num_max - num_min) * random.random() for i in range(0, num_var)]
        pop.obj = function(num_obj, pop.var)
        Pop.append(pop)
    Pop = crowding_distance_assiginment(Pop)
    return Pop

In [3]:
import math

"""
DTLZI测试函数
计算公式来源《多目标进化算法及其应用》-郑金华·著 [p190 - p191]
"""
def DTLZ1(M, X):
    """
    :param M:  多目标问题中的目标数
    :param X: 多目标问题中的自变量
    :return: 每一个目标的值
    """
    n = len(X)
    assert (n > M), 'The number of variables {0} cannot be less than the object number {1}'.format(n, M)
    k = n - M + 1
    f = []
    X_M = X[len(X)-k:len(X)]
    cont = 0.5 * g(k,X_M)
    for i in range(0,M):
        num = M - 1 - i
        res = cont
        for j in range(0,num):
            res *= X[j]
        if i != 0:
            res *= (1 - X[num])
        f.append(res)
    return f

def g(k,X_M):
    """
    :param k: 决策变量的后k个变量数
    :param X_M: 决策变量的后k个变量
    :return: 返回g(X_M)的计算结果
    """
    return 100 * ( k + sum([(X_M[i] - 0.5)**2 - math.cos(20 * math.pi * (X_M[i] - 0.5)) for i in range(0,k)]))

In [4]:
def pareto(P_1, P_2):
    """
    :param P_1: 个体1的目标值序列
    :param P_2: 个体2的目标值序列
    :return: P_1支配P_2返回true，否则返回false
    """
    assert (len(P_1) == len(P_2)), 'The object number of individual must equal P_1_len = {0} P_2_len = {1}'.format(len(P_1), len(P_1))
    lenght = len(P_1)
    a = b = 0
    for i in range(0,lenght):
        if P_1[i] < P_2[i]:
            a += 1
        if P_1[i] > P_2[i]:
            b += 1
    return (a > 0 and b == 0)

In [5]:
def non_dominated_sort(Pop):
    """
    :param Pop: 种群中每个个体的目标值序列
    :return:返回排序后的各层种群集合
    """
    Pop = init_step(Pop)
    pop_size = len(Pop)
    """
    变量说明：
    P[i]分层后每一层的个体集合
    """
    P = [[] for i in range(0, pop_size)]
    for p in Pop:
        for q in Pop:
            if pareto(p.obj, q.obj):
                p.s.append(q)
            elif pareto(q.obj, p.obj):
                p.n += 1
        # end for q
        if p.n == 0:
            P[0].append(p)
    # end for p
    i = 0
    while(len(P[i]) != 0):
        H = []
        for p in P[i]:
            for q in p.s:
                q.n -= 1
                if q.n == 0:
                    q.rank = i+1
                    H.append(q)
        # end for p
        i += 1
        P[i] = H
    # end for while
    return P
#end for sort

In [6]:
def crowding_distance_assiginment(P):
    N = len(P)
    for i in P:
        i.distance = 0
    for m in range(len(P[0].obj)):
        P = sorted(P, key = lambda p : p.obj[m])
        for i in range(2,N-1):
            P[i].distance += (P[i+1].obj[m] - P[i-1].obj[m])
    # end for m
    P[0].distance = P[N-1].distance = float('inf')
    return P

In [7]:
def binary_tournament_selection(Pop):
    pop_size = len(Pop) - 1
    rand1 = random.randint(0, pop_size)
    rand2 = random.randint(0, pop_size)
    while(rand1 == rand2):
        rand1 = random.randint(0, pop_size)
        rand2 = random.randint(0, pop_size)
    if Pop[rand1].rank < Pop[rand2].rank:
        return Pop[rand1]
    elif Pop[rand2].rank < Pop[rand1].rank:
        return Pop[rand2]
    else:
        if Pop[rand1].distance > Pop[rand2].distance:
            return Pop[rand1]
        else:
            return Pop[rand2]

def crossover(Pop, function):
    lenght = len(Pop)
    Q = []
    i = 0
    while (len(Q) < lenght):
        p1 = binary_tournament_selection(Pop)
        p2 = binary_tournament_selection(Pop)
#         p1 = Pop[i]
#         p2 = Pop[i+1]
        temp = p1.var[int(len(p1.var) / 2):]
        p1.var[int(len(p1.var) / 2):] = p2.var[int(len(p1.var) / 2):]
        p2.var[int(len(p1.var) / 2):] = temp
        p1.var = mutation(p1.var)
        p2.var = mutation(p2.var)
        Q.append(p1)
        Q.append(p2)
        i += 2
    return init_step(Q)

def mutation(var):
    lenght = len(var)
    for i in range(lenght):
        if random.random() > 0.1:
            var[i] = random.random()
    return var

def make_new_pop(Pop, function):
    return crossover(Pop,function)

In [8]:
def stretch(Pop):
    P_temp = []
    for p in Pop:
        if len(p) == 0:
            break
        while(len(p) != 0):
            P_temp.append(p.pop(0))
    return P_temp

def U(P, Q):
    R = []
    for p in P:
        R.append(p)
    for q in Q:
        R.append(q)
    return R

def init_step(Pop):
    for p in Pop:
        p.n = 0
        p.s = []
        p.obj = DTLZ1(num_obj, p.var)
    Pop = crowding_distance_assiginment(Pop)
    return Pop

In [9]:
# 初始化参数
pop_size = 1000
global num_obj
num_obj = 2
global num_var
num_var = 5
num_max = 1.0
num_min = 0.0
num_gene = 100

In [10]:
P = [[] for i in range(num_gene)]
Q = [[] for i in range(num_gene)]
R = [[] for i in range(num_gene)]

P[0] = init_pop(pop_size, num_var, num_obj, num_max, num_min, DTLZ1)


P[0] = non_dominated_sort(P[0])
P[0] = stretch(P[0])

Q[0] = make_new_pop(P[0], DTLZ1)

R[0] = U(P[0], Q[0])

R[0] = non_dominated_sort(P[0])

R[0] = stretch(R[0])

now_generation = 1


while(now_generation < num_gene):
    P[now_generation] = sorted(R[now_generation - 1], key = lambda x :(x.rank,-x.distance))[:pop_size]
    
    print(P[now_generation][0].obj)
    
    P[now_generation] = init_step(P[now_generation])

    P[now_generation] = non_dominated_sort(P[now_generation])

    P[now_generation] = stretch(P[now_generation])
    
    Q[now_generation] = make_new_pop(P[now_generation], DTLZ1)
    

    R[now_generation] = U(P[now_generation], Q[now_generation])
    now_generation += 1
    

[0.1062329296987122, 238.49925626208721]
[39.46520171448563, 251.83944693034695]
[253.71722551108954, 7.5225496152182005]
[303.9533394622211, 60.26745428936207]
[86.80630178070868, 325.15855938866656]
[262.31084579249176, 35.13871925774113]
[177.19778498830644, 77.76139157794985]
[323.63095877832075, 21.453938520420255]
[328.04313684953144, 36.254865717135345]
[3.9274912339507626, 316.05114546587447]
[276.2989799731751, 1.2633801663855229]
[259.6256757258492, 2.266131128412633]
[24.094645130999286, 308.6804217215285]
[279.3098347833425, 9.076486048165318]
[20.1542634778014, 329.168568297707]
[74.00454356867726, 329.0052712285538]
[10.64391199292646, 285.83720549470615]
[18.52680147889654, 366.78669286074785]
[9.502157517582525, 322.35969098362085]
[298.92943125942145, 32.989398511235635]
[211.60004419595893, 0.6280471775567357]
[210.62776922878533, 0.4589273310991014]
[5.960898910627311, 260.5314162258376]
[3.1730215363239327, 226.94120277828935]
[44.28198246510425, 269.4749000913586]
